### 0. Install Julia and POMDP.jl

Please first install Julia and POMDP.jl (https://github.com/JuliaPOMDP/POMDPs.jl).

### 1. Include Julia packages and load the problem
Several POMDP julia packages such as "POMDPTools, POMDPModels" are already included in planner.jl.

If there is an error about uninstalled packages, please install them with "pkg.add()".

In [ ]:
include("./NaivePOMCGS/Planner.jl")
using Plots
using RockSample

pomdp = RockSamplePOMDP(15,15)
pb_name = "RS1515"
action_space = collect(actions(pomdp))
b0 = initialstate(pomdp)

### 2. Define parameters

In [ ]:
max_planning_secs = 20000
nb_process_samples = 20000
c = 2.0
max_belief_gap = 0.3
epsilon = 0.01
nb_runs = 10

## 3. Repeatedly Evaluate POMCGS (10 runs)

In [ ]:
for i in 1:nb_runs
    # Define Q learning
    Q_table = Dict{Any, Dict{Int64, Float64}}()
    V_table = Dict{Any, Float64}()
    learning_rate = 0.9
    explore_rate = 0.65
    nb_particles_b0 = nb_process_samples
    b0_particles = []
    for i in 1:nb_particles_b0
        push!(b0_particles, rand(b0))
    end
    Q_learning_policy = Qlearning(Q_table, V_table, learning_rate, explore_rate, action_space, typemin(Float64), typemax(Float64))
    RL = FindRLower(pomdp, b0, action_space)
    
    # Train Q-learning
    nb_episode_size = 10
    nb_max_episode = 10
    nb_sim = 20
    epsilon_Q_learning = 0.1
    Training(Q_learning_policy, nb_episode_size, nb_max_episode, nb_sim, epsilon_Q_learning, b0_particles, pomdp)

    # Init Planner
    planner = NaivePlannerPOMCGS(RL, Q_learning_policy)
    planner._max_planning_secs = max_planning_secs
    InitPlannerParameters(planner, nb_process_samples, max_belief_gap, c, discount(pomdp), epsilon)
    particles_b0, dict_weighted_b0, action_space = InitPOMDP(pomdp, b0, nb_particles_b0)
    fsc = InitFSC(max_belief_gap, planner._max_graph_node_size, action_space)

    # POMCG's main search  
    Search(pomdp, particles_b0, dict_weighted_b0, fsc, planner)

    # Export Results
    result_nodes_list = Prunning(fsc)
    final_fsc_length = length(result_nodes_list)
    outname = pb_name*string.(final_fsc_length)*"nbsamples"*string.(nb_process_samples)
    ExportLogData(planner, outname)
end